# Task 1 d, Calculating strength reserve factor of all elements

## Please enter your name (daniel, fabian, felix, yannis)

In [ ]:
name = "yannis"

## Imports 

In [2]:
import pandas as pd
import sys 
import os

sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *


In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

## File Selection 

In [4]:
#Import the stress of the panels (OLD VERSION)
# paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)

In [5]:
# This will become the new import for the panel stresses
paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = add_component_names_to_elements(paneldf_v2, element_component_df)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)
paneldf_v2.reset_index(drop=True, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True) # @yannisendr this is possible, since pandas allows non-unique indices, however operations that require an unique index won't work 

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

# copy the new df to the previous one
paneldf = paneldf_v2
#print(paneldf.to_string())
paneldf_v2.head(10)

,Load Case,sigmaXX,sigmaXY,sigmaYY,sigmaYZ,sigmaXZ,sigmaZZ,Component Name
Element ID,,,,,,,,
1,Subcase 1 (LC1),-66.67,26.25,12.75,0.0,0.0,0.0,panel1
2,Subcase 1 (LC1),-82.97,27.10,20.87,0.0,0.0,0.0,panel1
3,Subcase 1 (LC1),-67.63,36.97,20.74,0.0,0.0,0.0,panel1
4,Subcase 1 (LC1),-97.67,34.52,2.42,0.0,0.0,0.0,panel2
5,Subcase 1 (LC1),-89.50,27.04,25.35,0.0,0.0,0.0,panel2
6,Subcase 1 (LC1),-96.29,28.46,26.60,0.0,0.0,0.0,panel2
7,Subcase 1 (LC1),-94.14,33.47,-2.96,0.0,0.0,0.0,panel3
8,Subcase 1 (LC1),-96.16,26.79,20.02,0.0,0.0,0.0,panel3
9,Subcase 1 (LC1),-93.57,29.76,37.30,0.0,0.0,0.0,panel3


In [6]:
# Import the stress of the stringers 
#stringerdf = pd.read_excel(f'../data/{name}/stringer.xlsx', index_col=0)

stringerdf_v2 = pd.read_csv(f'../data/{name}/stringer_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

stringerdf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
for index, row in stringerdf_v2.iterrows():
    element_id = row['Element ID']
    # Find the matching row in the element_component_df
    matching_row = element_component_df[element_component_df['Element ID'] == element_id]
    if not matching_row.empty:
        component = matching_row['Component Name'].values[0]
        stringerdf_v2.at[index, 'Component Name'] = component
    else:
        stringerdf_v2.at[index, 'Component Name'] = None  # or handle missing components as needed

stringerdf_v2.set_index('Element ID', inplace=True)

stringerdf_v2.rename(columns={'Element Stresses (1D):CBAR/CBEAM Axial Stress': 'sigmaXX', 'Loadcase': 'Load Case'}, inplace=True)
stringerdf_v2.drop(columns=['FileID', 'Step'], inplace=True)

stringerdf_v2['Load Case'] = stringerdf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

#print(stringerdf.head(30))
stringerdf_v2.head(10)

stringerdf = stringerdf_v2


## Constants 

In [7]:
sigma_ult = 530

## Applying Functions 

In [8]:
#Adding reserve Factor column to the panels 
paneldf['reserveFactor'] = paneldf.apply(panelStrength_calc, sigma_ult= sigma_ult, axis=1)
paneldf.head(10)

,Load Case,sigmaXX,sigmaXY,sigmaYY,sigmaYZ,sigmaXZ,sigmaZZ,Component Name,reserveFactor
Element ID,,,,,,,,,
1,Subcase 1 (LC1),-66.67,26.25,12.75,0.0,0.0,0.0,panel1,4.073245
2,Subcase 1 (LC1),-82.97,27.10,20.87,0.0,0.0,0.0,panel1,3.330609
3,Subcase 1 (LC1),-67.63,36.97,20.74,0.0,0.0,0.0,panel1,3.447039
4,Subcase 1 (LC1),-97.67,34.52,2.42,0.0,0.0,0.0,panel2,3.057297
5,Subcase 1 (LC1),-89.50,27.04,25.35,0.0,0.0,0.0,panel2,3.085300
6,Subcase 1 (LC1),-96.29,28.46,26.60,0.0,0.0,0.0,panel2,2.887791
7,Subcase 1 (LC1),-94.14,33.47,-2.96,0.0,0.0,0.0,panel3,3.231790
8,Subcase 1 (LC1),-96.16,26.79,20.02,0.0,0.0,0.0,panel3,3.015894
9,Subcase 1 (LC1),-93.57,29.76,37.30,0.0,0.0,0.0,panel3,2.768049


In [9]:
#Adding reserve Factor column to the stringers 
stringerdf['reserveFactor'] = stringerdf.apply(stringerStrength_calc, sigma_ult= sigma_ult, axis=1)
stringerdf.head(10)

,Load Case,sigmaXX,Component Name,reserveFactor
Element ID,,,,
40,Subcase 1 (LC1),-77.67,stringer1,4.549161
41,Subcase 1 (LC1),-70.60,stringer1,5.004721
42,Subcase 1 (LC1),-79.68,stringer1,4.434404
43,Subcase 1 (LC1),-81.30,stringer2,4.346043
44,Subcase 1 (LC1),-85.24,stringer2,4.145159
45,Subcase 1 (LC1),-85.16,stringer2,4.149053
46,Subcase 1 (LC1),-81.42,stringer3,4.339638
47,Subcase 1 (LC1),-83.91,stringer3,4.210861
48,Subcase 1 (LC1),-85.17,stringer3,4.148566


# Making things pretty 

## Delete unnecessary columns 

In [10]:
# First for the stringers 
stringerdf = stringerdf.drop(['Component Name','sigmaXX'], axis=1)
stringerdf

,Load Case,reserveFactor
Element ID,,
40,Subcase 1 (LC1),4.549161
41,Subcase 1 (LC1),5.004721
42,Subcase 1 (LC1),4.434404
43,Subcase 1 (LC1),4.346043
44,Subcase 1 (LC1),4.145159
...,...,...
62,Subcase 3 (LC3),8.290318
63,Subcase 3 (LC3),11.766012
64,Subcase 3 (LC3),7.787819


In [11]:
# Then also the panels 
paneldf=paneldf.drop(['Component Name', 'sigmaXX', 'sigmaYY', 'sigmaXY'], axis = 1)
paneldf.head(10)

,Load Case,sigmaYZ,sigmaXZ,sigmaZZ,reserveFactor
Element ID,,,,,
1,Subcase 1 (LC1),0.0,0.0,0.0,4.073245
2,Subcase 1 (LC1),0.0,0.0,0.0,3.330609
3,Subcase 1 (LC1),0.0,0.0,0.0,3.447039
4,Subcase 1 (LC1),0.0,0.0,0.0,3.057297
5,Subcase 1 (LC1),0.0,0.0,0.0,3.085300
6,Subcase 1 (LC1),0.0,0.0,0.0,2.887791
7,Subcase 1 (LC1),0.0,0.0,0.0,3.231790
8,Subcase 1 (LC1),0.0,0.0,0.0,3.015894
9,Subcase 1 (LC1),0.0,0.0,0.0,2.768049


## Concat data frames beneth each other 

In [12]:
combined = pd.concat([paneldf,stringerdf],axis=0)
combined.sample(20)

,Load Case,sigmaYZ,sigmaXZ,sigmaZZ,reserveFactor
Element ID,,,,,
15,Subcase 3 (LC3),0.0,0.0,0.0,0.579777
1,Subcase 2 (LC2),0.0,0.0,0.0,6.908312
57,Subcase 3 (LC3),NaN,NaN,NaN,8.685677
45,Subcase 2 (LC2),NaN,NaN,NaN,6.563874
25,Subcase 2 (LC2),0.0,0.0,0.0,5.079081
40,Subcase 2 (LC2),NaN,NaN,NaN,6.284833
65,Subcase 3 (LC3),NaN,NaN,NaN,10.009443
56,Subcase 3 (LC3),NaN,NaN,NaN,8.505858
46,Subcase 2 (LC2),NaN,NaN,NaN,6.934904


## Split into three seperate loadcase data frames 

In [13]:
loadCase1df = combined[combined['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = combined[combined['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = combined[combined['Load Case'] == 'Subcase 3 (LC3)']
loadCase1df.head(20)

,Load Case,sigmaYZ,sigmaXZ,sigmaZZ,reserveFactor
Element ID,,,,,
1,Subcase 1 (LC1),0.0,0.0,0.0,4.073245
2,Subcase 1 (LC1),0.0,0.0,0.0,3.330609
3,Subcase 1 (LC1),0.0,0.0,0.0,3.447039
4,Subcase 1 (LC1),0.0,0.0,0.0,3.057297
5,Subcase 1 (LC1),0.0,0.0,0.0,3.085300
6,Subcase 1 (LC1),0.0,0.0,0.0,2.887791
7,Subcase 1 (LC1),0.0,0.0,0.0,3.231790
8,Subcase 1 (LC1),0.0,0.0,0.0,3.015894
9,Subcase 1 (LC1),0.0,0.0,0.0,2.768049


In [14]:
# Drop the loadcase columns 
loadCase1df = loadCase1df.drop(['Load Case'], axis = 1)
loadCase2df = loadCase2df.drop(['Load Case'], axis = 1)
loadCase3df = loadCase3df.drop(['Load Case'], axis = 1)
#Rename the reserveFactor column 
loadCase1df = loadCase1df.rename(columns={"reserveFactor": "Load Case 1 RF"})
loadCase2df = loadCase2df.rename(columns={"reserveFactor": "Load Case 2 RF"})
loadCase3df = loadCase3df.rename(columns={"reserveFactor": "Load Case 3 RF"})
loadCase2df.tail(5)

,sigmaYZ,sigmaXZ,sigmaZZ,Load Case 2 RF
Element ID,,,,
62,NaN,NaN,NaN,6.548060
63,NaN,NaN,NaN,6.891620
64,NaN,NaN,NaN,6.139589
65,NaN,NaN,NaN,5.984643
66,NaN,NaN,NaN,6.284833


In [15]:
# Now we concat them back together 
outputdf = pd.concat([loadCase1df,loadCase2df, loadCase3df], axis=1)
outputdf

,sigmaYZ,sigmaXZ,sigmaZZ,Load Case 1 RF,sigmaYZ,sigmaXZ,sigmaZZ,Load Case 2 RF,sigmaYZ,sigmaXZ,sigmaZZ,Load Case 3 RF
Element ID,,,,,,,,,,,,
1,0.0,0.0,0.0,4.073245,0.0,0.0,0.0,6.908312,0.0,0.0,0.0,1.087410
2,0.0,0.0,0.0,3.330609,0.0,0.0,0.0,5.930498,0.0,0.0,0.0,1.237222
3,0.0,0.0,0.0,3.447039,0.0,0.0,0.0,6.019281,0.0,0.0,0.0,1.044074
4,0.0,0.0,0.0,3.057297,0.0,0.0,0.0,5.174529,0.0,0.0,0.0,1.053799
5,0.0,0.0,0.0,3.085300,0.0,0.0,0.0,5.600788,0.0,0.0,0.0,1.236043
6,0.0,0.0,0.0,2.887791,0.0,0.0,0.0,5.079081,0.0,0.0,0.0,1.031814
7,0.0,0.0,0.0,3.231790,0.0,0.0,0.0,5.304930,0.0,0.0,0.0,1.026340
8,0.0,0.0,0.0,3.015894,0.0,0.0,0.0,5.298586,0.0,0.0,0.0,1.245921
9,0.0,0.0,0.0,2.768049,0.0,0.0,0.0,4.932736,0.0,0.0,0.0,0.974209


# ROUND & Export back to Excel 

In [16]:
outputdf = outputdf.round(rounding_digits)
outputdf.to_excel(f'../data/{name}/output/processed_d.xlsx')
outputdf.head(10)

,sigmaYZ,sigmaXZ,sigmaZZ,Load Case 1 RF,sigmaYZ,sigmaXZ,sigmaZZ,Load Case 2 RF,sigmaYZ,sigmaXZ,sigmaZZ,Load Case 3 RF
Element ID,,,,,,,,,,,,
1,0.0,0.0,0.0,4.07,0.0,0.0,0.0,6.91,0.0,0.0,0.0,1.09
2,0.0,0.0,0.0,3.33,0.0,0.0,0.0,5.93,0.0,0.0,0.0,1.24
3,0.0,0.0,0.0,3.45,0.0,0.0,0.0,6.02,0.0,0.0,0.0,1.04
4,0.0,0.0,0.0,3.06,0.0,0.0,0.0,5.17,0.0,0.0,0.0,1.05
5,0.0,0.0,0.0,3.09,0.0,0.0,0.0,5.60,0.0,0.0,0.0,1.24
6,0.0,0.0,0.0,2.89,0.0,0.0,0.0,5.08,0.0,0.0,0.0,1.03
7,0.0,0.0,0.0,3.23,0.0,0.0,0.0,5.30,0.0,0.0,0.0,1.03
8,0.0,0.0,0.0,3.02,0.0,0.0,0.0,5.30,0.0,0.0,0.0,1.25
9,0.0,0.0,0.0,2.77,0.0,0.0,0.0,4.93,0.0,0.0,0.0,0.97
